In [3]:
import pandas as pd
import numpy as np
import polars as pl
import polars.selectors as cs
import sys, os
import pickle as pkl
import dproc
from functools import partial

print(sys.version)

for i in [pd, pl, np]:
    print(i.__name__, i.__version__)

3.12.3 (main, May  7 2024, 08:28:12) [GCC 9.4.0]
pandas 2.2.2
polars 0.20.24
numpy 1.26.4


# Dataset Description

The dataset for this competition (both train and test) was generated from a deep learning model trained on the [Flood Prediction Factors dataset](https://www.kaggle.com/datasets/brijlaldhankour/flood-prediction-factors). Feature distributions are close to, but not exactly the same, as the original. Feel free to use the original dataset as part of this competition, both to explore differences as well as to see whether incorporating the original in training improves model performance.

Note: This dataset is particularly well suited for visualizations, clustering, and general EDA. Show off your skills!

In [4]:
data_path = 'data'
if not os.path.isdir(data_path):
    os.mkdir(data_path)
files = {
    k: os.path.join(p, f)
    for k, p, f in [
        ('train', data_path, 'train.csv'),
        ('org', data_path, 'flood.csv'),
        ('test', data_path, 'test.csv'),
        ('type', data_path, 'type.pkl'),
        ('train_pkl', data_path, 'train.pkl'),
        ('org_pkl', data_path, 'org.pkl'),
        ('test_pkl', data_path, 'test.pkl'),
        ('var_pkl', data_path, 'var.pkl')
    ]
}

if not os.path.isfile(files['train']):
    !kaggle competitions download -c playground-series-s4e5
    !unzip playground-series-s4e5.zip -d data
    !rm playground-series-s4e5.zip
    !kaggle datasets download -d brijlaldhankour/flood-prediction-factors
    !unzip flood-prediction-factors.zip -d data
    !rm flood-prediction-factors.zip

## 변수 정의

In [5]:
df_var = pd.DataFrame([
    ['id', "Index"],
    ['FloodProbability', "target: 홍수 확률"],
    ['AgriculturalPractices', ""],
    ['ClimateChange', ""],
    ['CoastalVulnerability', ""],
    ['DamsQuality', ""],
    ['Deforestation', ""],
    ['DeterioratingInfrastructure', ""],
    ['DrainageSystems', ""],
    ['Encroachments', ""],
    ['InadequatePlanning', ""],
    ['IneffectiveDisasterPreparedness', ""],
    ['Landslides', ""],
    ['MonsoonIntensity', ""],
    ['PoliticalFactors', ""],
    ['PopulationScore', ""],
    ['RiverManagement', ""],
    ['Siltation', ""],
    ['TopographyDrainage', ""],
    ['Urbanization', ""],
    ['Watersheds', ""],
    ['WetlandLoss', ""]
], columns=['name', 'Description']).assign(src='org').set_index('name')
df_var

,Description,src
name,,
id,Index,org
FloodProbability,target: 홍수 확률,org
AgriculturalPractices,,org
ClimateChange,,org
CoastalVulnerability,,org
DamsQuality,,org
Deforestation,,org
DeterioratingInfrastructure,,org
DrainageSystems,,org


# 데이터 불러오기

## 변수 타입 설정

In [6]:
# train, org, test에서 적정 타입을 파악하기 위한 항목을 수집합니다.
df_type = dproc.merge_type_df([
    pl.scan_csv(files[i]).pipe(dproc.get_type_df)
    for i in ['train', 'org', 'test']
])
df_type

,min,max,na,count,n_unique,dtype,f32,i32,i16,i8
feature,,,,,,,,,,
AgriculturalPractices,0.000,16.000,0.0,1913262.0,17.000000,Int64,True,True,True,True
ClimateChange,0.000,17.000,0.0,1913262.0,18.000000,Int64,True,True,True,True
CoastalVulnerability,0.000,17.000,0.0,1913262.0,18.000000,Int64,True,True,True,True
DamsQuality,0.000,16.000,0.0,1913262.0,17.000000,Int64,True,True,True,True
Deforestation,0.000,17.000,0.0,1913262.0,18.000000,Int64,True,True,True,True
DeterioratingInfrastructure,0.000,17.000,0.0,1913262.0,18.000000,Int64,True,True,True,True
DrainageSystems,0.000,17.000,0.0,1913262.0,18.000000,Int64,True,True,True,True
Encroachments,0.000,18.000,0.0,1913262.0,18.333333,Int64,True,True,True,True
FloodProbability,0.285,0.725,0.0,1167957.0,83.000000,Float64,True,True,True,True


In [7]:
# index로 판단되는 id는 Int64로 하고 나머지는 자동으로 타입을 정합니다.
dtypes = dproc.get_type_pl(df_type, predefine={'id': pl.Int64})
# type을 df_feature에 포함 시킵니다.
df_var = df_var.join(pd.Series(dtypes, name='type').apply(lambda x: str(x)))

[Sorting along the feature axis as "feature engineering"](https://www.kaggle.com/competitions/playground-series-s4e5/discussion/499579)

각 변수별 소트한 결과가 의미있는 성능 향상을 보일 수 있다고 합니다.

In [8]:
dfl_train = pl.read_csv(files['train'], dtypes=dtypes)
dprocs = list()

In [9]:
X_num_org= df_var.loc[~df_var.index.isin(['id', 'FloodProbability']) & (df_var['src'] == 'org')].index.tolist()
row_wise_sort = lambda x: x.select(
    pl.concat_list(cs.by_name(X_num_org)).list.sort().list.to_struct().alias('s')
).unnest('s').rename(lambda x: 'sort' + '_'+ x.split('_')[1])

dprocs.append(partial(
    dproc.select_opr, 
    select_proc=row_wise_sort, 
    desc=[('sort_{}'.format(i), '원래 입력변수 중에서 {} 번째로 작은 수'.format(i + 1)) for i in range(len(X_num_org))],
    src='sort'
))

[PSS4E5 EDA which makes sense ⭐️⭐️⭐️⭐️⭐️](https://www.kaggle.com/code/ambrosm/pss4e5-eda-which-makes-sense) 에서, 

변수의 합 만으로도 연관성이 속성을 만들어 낼 수 있다고 합니다. 

먼저 선형 회귀 모델을 만들어보고 잔차를 출력해 봅니다

In [10]:
with_columns = [
    ('s_values', 's_values', pl.fold(acc=pl.lit(0), function=lambda acc, x: acc + x, exprs=cs.by_name(X_num_org)), "제공된 모든변수의 합을 만듭니다."),
    ('s_values', 's_values2', pl.fold(acc=pl.lit(0), function=lambda acc, x: acc + (x ** 2), exprs=cs.by_name(X_num_org)), "제공된 모든변수의 제곱 수의 합을 만듭니다.")
]
dprocs.append(partial(dproc.with_columns_opr, proc_list=with_columns))

[PSS4E5 EDA which makes sense ⭐️⭐️⭐️⭐️⭐️](https://www.kaggle.com/code/ambrosm/pss4e5-eda-which-makes-sense)의 Advanced models with feature engineering에서
제안한 속성 추가.

In [13]:
dprocs.append(
    partial(dproc.with_columns_opr, 
            proc_list=[('special', 'special1', pl.col('s_values').is_in([72, 73, 74, 75]), 
                     's_values와 floodprobability와 선형 관계에 의한 floodprobability 예상치 보다 유달리 높은 구간에 속하는 여부')]
    )
)

In [14]:
dfl_train, df_var = dproc.apply_procs(dfl_train, dprocs, df_var)

# 데이터 저장

In [15]:
df_train = dfl_train.to_pandas().set_index('id')
del dfl_train
df_train.to_pickle(files['train_pkl'])

In [16]:
dfl_org = pl.read_csv(files['org'], dtypes=dtypes)
dfl_org = dfl_org.with_columns(id=-pl.int_range(0, pl.len()) - 1)
dfl_org, _ = dproc.apply_procs(dfl_org, dprocs)

df_org = dfl_org.to_pandas().set_index('id')
del dfl_org
df_org.to_pickle(files['org_pkl'])

In [17]:
dfl_test = pl.read_csv(files['test'], dtypes=dtypes)
dfl_test, _ = dproc.apply_procs(dfl_test, dprocs)

df_test = dfl_test.to_pandas().set_index('id')
del dfl_test
df_test.to_pickle(files['test_pkl'])

In [18]:
df_var.to_pickle(files['var_pkl'])